<center> <h2> <b> Light AutoML </b> </h2></center>

This notebook is starter code to explore automl and optuna models, more tweekings can improve scores. Will try experiment with more packages.  
Following AutoML and Optuna models are tried:
<ol type="1">
<li>Player-1 (Version 5): <b>H2O AutoML</b> </li>
<li>Player-2 (Version 6): <b>Auto Keras</b></li>
<li>Player-3 (Version 9): <b>Auto Glucon</b></li>
<li>Player-4 (Version 13): <b>XGBoost with Optuna</b></li>   
<li>Player-5 (Version 14): <b>CatBoost with Optuna</b></li>  
<li>Player-6 (Version 15): <b>TPOT</b></li> 
<li>Player-7 (Version 18): <b>Light AutoML</b></li>
</ol>  

This notebook is completely inspired from approach by <b>Alexander Ryzhkov's</b> <a href="https://www.kaggle.com/alexryzhkov/lightautoml-baseline-tps-june-2021">notebook</a> and <b>Laurent Pourchot's</b> <a href="https://www.kaggle.com/pourchot/simple-neural-network/notebook"> notebook</a>

In [ ]:
!pip install -U lightautoml >> /dev/null

In [ ]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import NumericRole


import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
N_THREADS = 4 
N_FOLDS = 5 
RANDOM_STATE = 42 
TEST_SIZE = 0.2 
TIMEOUT = 2*3600 
TARGET_NAME = 'target'

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
targets = pd.get_dummies(train_data[TARGET_NAME])

In [ ]:
test_data = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

In [ ]:
cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [ ]:
def conv_model():

    conv_inputs = layers.Input(shape = (75))
    #----------- Embedding layers ----------------------
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    #----------- Convolution layers ----------------------
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    #----------- Residual blocks layers ----------------------
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'elu',
                kernel_initializer = "lecun_normal"))(output)
    
    #----------- Final layer -----------------------
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    #----------- Model instantiation  ---------------
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [ ]:
oof_NN_a = np.zeros((train_data.shape[0],9))
pred_NN_a = np.zeros((test_data.shape[0],9))

N_FOLDS = 20
SEED = 2021
EPOCH = 50


skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, ts_idx) in enumerate(skf.split(train_data,train_data.iloc[:,-1])):
    print(f"\n ====== TRAINING FOLD {fold} =======\n")

    X_train = train_data.iloc[:,1:-1].iloc[tr_idx]
    y_train = targets.iloc[tr_idx]
    X_test = train_data.iloc[:,1:-1].iloc[ts_idx]
    y_test = targets.iloc[ts_idx]

    K.clear_session()

    #================= NN CONV MODEL training =========
    
    print("\n-----Convolution model Training----\n")

    model_conv = conv_model()

    model_conv.compile(loss='categorical_crossentropy', 
                            optimizer = keras.optimizers.Adam(learning_rate=2e-4), 
                            metrics=custom_metric)
    model_conv.fit(X_train, y_train,
              batch_size = 256, epochs = EPOCH,
              validation_data=(X_test, y_test),
              callbacks=[es, plateau],
              verbose = 0)
   
    #============== Convolution Model prediction ==========

    pred_a = model_conv.predict(X_test) 
    oof_NN_a[ts_idx] += pred_a 
    score_NN_a = log_loss(y_test, pred_a)
    print(f"\nFOLD {fold} Score convolution model: {score_NN_a}\n")
    pred_NN_a += model_conv.predict(test_data.iloc[:,1:]) / N_FOLDS 


score_a = log_loss(targets, oof_NN_a)
print(f"\n=== FINAL SCORE CONVOLUTION MODEL : {score_a}===\n") 

In [ ]:
cols = [f'Class_{x+1}' for x in range(9)]

In [ ]:
OOF_PREDS_NN = pd.DataFrame(oof_NN_a, columns=cols)
TEST_PREDS_NN = pd.DataFrame(pred_NN_a, columns=cols)

In [ ]:
for col in cols:
    train_data[col] = OOF_PREDS_NN[col]
    test_data[col] = TEST_PREDS_NN[col]

In [ ]:
%%time

task = Task('multiclass',metric = custom_metric, greater_is_better = False)

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

In [ ]:
%%time 

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
    NumericRole(np.float32, prob = True): cols
}

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       cpu_limit = N_THREADS,
                       general_params = {
                           'use_algos': [['lgb_tuned', 'cb_tuned']],
                       },
                       tuning_params = {'max_tuning_time': 1800},
                       reader_params = {'n_jobs': N_THREADS},
                       configs_list = ['../input/lightautoml-configs/conf_1_sel_type_1.yml'],
                       max_runs_per_config=1
                       )
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

In [ ]:
test_pred = automl.predict(test_data)

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('TPS_June_lightautoml_v5.csv', index = False)